In [1]:
from Dexined_model import *
from PIL import Image
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
Path.ls = lambda x: list(x.iterdir())

In [44]:
src_imgs = Path().home()/'Datasets/buildings_sketch_dataset/Source/photos/pin_contempo2'
len(src_imgs.ls())

202

In [3]:
src_saved = Path().home()/'0-trained_models'
src_saved.ls()

[PosixPath('/Users/antoninsumner/0-trained_models/dexined-100e-pxshuff.pth'),
 PosixPath('/Users/antoninsumner/0-trained_models/dexined-100e-newloss.pth'),
 PosixPath('/Users/antoninsumner/0-trained_models/rednet-50e.pth')]

In [4]:
p = Path().home().absolute()/'Datasets/buildings_sketch_dataset/dataset'

# Load DexiNed pretrained model

In [5]:
dexined_model = DexiNet(up_mode='pixel_shuffle')

In [6]:
state = torch.load(src_saved/'dexined-100e-newloss.pth', map_location=torch.device('cpu'))['model']

In [7]:
dexined_model.load_state_dict(state)

<All keys matched successfully>

In [8]:
dexined_model.eval()
print()

# Utils

In [19]:
def resize_same(img, size=512):
    sz = np.array(img.size)
    r = sz[-1] / sz[0]
    nsz = [int(size * r**(-sz.argmax())), int(size * r**(1-sz.argmax()))]
    return img.resize(nsz)

In [38]:
def batch_one(img):
    res = np.array(img)
    print('array',res.shape)
    res = torch.from_numpy(res).float()/255
    if len(res.shape)==2: res = torch.stack(3*[res], dim=-1) 
    res = res.permute(2,0,1)
    if res.shape[0]>3: res = res[:3,...]
    res = torch.stack(2*[res])
    print('Batch',res.shape)
    return res

In [39]:
def norm_tensor(x):
    y = (x-x.min())/(x.max()-x.min())
    y *= 255
    return y

In [43]:
(src_imgs.ls()[0]).stem

'71ce73be63732a25dc5509ffd1b16397'

In [49]:
def make_preds(m, fns, dest, img_size=512):
    fns = [f for f in fns if not str(Path(f).name).startswith('.')]
    assert dest.exists()
    assert (dest/'imgs').exists()
    assert (dest/'edgemaps').exists()
    n =  max([int(f.stem) for f in (dest/'imgs').ls() if not str(f.stem).startswith('.')])
    for i in range(len(fns)):
        file = dest/f'imgs/{n+1+i}.png'
        edge_file = dest/f'edgemaps/{n+1+i}.png'
        print(file)
        if file.exists(): continue
        img = Image.open(fns[i])
        img = resize_same(img, size=img_size)
        img.save(str(file))
        
        b = batch_one(img)
        y = m(b)
        y = y[2][0,0,...].detach()
        y = norm_tensor(y)
        plt.imsave(str(edge_file), y, cmap='binary')


# Generate files

In [50]:
make_preds(dexined_model, src_imgs.ls(), p)

/Users/antoninsumner/Datasets/buildings_sketch_dataset/dataset/imgs/390.png
array (340, 512, 3)
Batch torch.Size([2, 3, 340, 512])
/Users/antoninsumner/Datasets/buildings_sketch_dataset/dataset/imgs/391.png
array (361, 512, 3)
Batch torch.Size([2, 3, 361, 512])
/Users/antoninsumner/Datasets/buildings_sketch_dataset/dataset/imgs/392.png
array (384, 512, 3)
Batch torch.Size([2, 3, 384, 512])
/Users/antoninsumner/Datasets/buildings_sketch_dataset/dataset/imgs/393.png
array (465, 512, 3)
Batch torch.Size([2, 3, 465, 512])
/Users/antoninsumner/Datasets/buildings_sketch_dataset/dataset/imgs/394.png
array (512, 358, 3)
Batch torch.Size([2, 3, 512, 358])
/Users/antoninsumner/Datasets/buildings_sketch_dataset/dataset/imgs/395.png
array (433, 512, 3)
Batch torch.Size([2, 3, 433, 512])
/Users/antoninsumner/Datasets/buildings_sketch_dataset/dataset/imgs/396.png
array (293, 512, 3)
Batch torch.Size([2, 3, 293, 512])
/Users/antoninsumner/Datasets/buildings_sketch_dataset/dataset/imgs/397.png
array (

OSError: cannot write mode CMYK as PNG